In [15]:
from pathlib import Path as pt
import numpy as np
import matplotlib.pyplot as plt

In [9]:
filename = pt("./22_10_20-1.thz")
filename.exists()

True

In [21]:
data = np.genfromtxt(filename)

In [23]:
with open(f"./{filename.stem}_corrected.thz", "w+") as f:
    for line in data:
        freq, c0, c1, *_ = line
        f.write(f"{freq:3f}\t{c0:3f}\t{c1:3f}\t\n")

In [ ]:
def mhz_wavenumber(freq):
    return freq*1e6/(speed_of_light*100)

mhz_wn = np.vectorize(mhz_wavenumber)

In [ ]:
from scipy.constants import m_p
fwhm = lambda freq, temp, m: np.sqrt( (8*Boltzmann*temp*np.log(2)) / (m*m_p*speed_of_light**2) ) * freq

freq = 117.692360 * 1e9
fwhm(freq, 5, 28)*1e-3

In [7]:
location = pt(r"./testFiles/measurements_THzScan")
location.exists()

True

In [32]:
thz_files = list(location.glob("*.txt"))

%matplotlib widget
fig, ax = plt.subplots(figsize=(12, 7), dpi=100)

for f in thz_files:
    x, y, *_ = np.genfromtxt(f).T
    if x[0]<118:
        ax.plot(x, y, ".", label=f.stem)

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
N_rates = solve_ivp(computeRateDistributionEquations, tspan, [*N, 0, 0], dense_output=True)

%matplotlib widget
simulateTime = np.linspace(0, t, 100)
simulateCounts = N_rates.sol(simulateTime)

fig, ax = plt.subplots(figsize=(7, 4), dpi=100)

ax.plot(simulateTime.T*1e-3, simulateCounts.T)
ax.plot(simulateTime*1e-3, simulateCounts.sum(axis=0), "k")

ax.set(yscale="log")

In [ ]:
def showRateDistributionEquations(totallevel, excitedLevel, lightON, includeSpontaneousEmissionForAllLevels, nHe="nHe"):
    
    fullRateEquation = []
    N = [f"N{i}" for i in range(totallevel)]
    N_He = [f"{nHe}{i}" for i in range(2)]
    
    rateCollection = []
    
    if lightON:
        temp0 = f"B_{excitedLevel-1}{excitedLevel}"
        temp1 = f"B_{excitedLevel}{excitedLevel-1}"

        # defined B rate from excited state 
        B_rate = f" + {temp0}*{N[excitedLevel-1]} - {temp1}*{N[excitedLevel]} "
    
    # For loop for combining different rates processes
    for i in range(totallevel):
        
        # Collisional rate
        rateEquationsCombined = []
        for j in range(totallevel):
            if i!= j: 
                
                key = f"q_{j}{i}"
                keyInverse = f"q_{i}{j}"
             
                k0 = f" + {key}*{nHe}*{N[j]} - {keyInverse}*{nHe}*{N[i]}"
                
                rateEquationsCombined.append(k0)
        
        
        # END: Collisional rate for loop
        
        # Einstein Coefficient A
        if i == 0:
            temp0 = f"A_10"
            k1 = f" + {temp0}*{N[1]}"
        
        if includeSpontaneousEmissionForAllLevels:
            if i == totallevel-1:
                temp0 = f"A_{i}{i-1}"
                k1 = f" - {temp0}*{N[i]}"

            if i>0 and i<totallevel-1:
                temp0 = f"A_{i}{i-1}"
                temp1 = f"A_{i+1}{i}"
                k1 = f" - {temp0}*{N[i]} + {temp1}*{N[i+1]}"
                
        elif i == 1:
            temp0 = f"A_10"
            k1 = f" - {temp0}*{N[1]}"
            
        # END: Einstein Coefficient A
        
        
        # Einstein Coefficient B
        # NOTE: B rate defined from excited state
        
        if lightON:
            if i==excitedLevel-1:
                k1 += f" - ({B_rate})"

            if i==excitedLevel:
                k1 += f"{B_rate}"
        
        # END: Einstein Coefficient B

        # Combining all equations into single array
        rateEquationsCombined.append(k1)
        rateCollection.append(rateEquationsCombined)
        
    # END: For loop for combining different rates processes
    
    # Computing all rates
    dR_dt = []

    for rates in rateCollection:
        temp = reduce(lambda a, b: a+b, rates)
        dR_dt.append(temp)
        
    # Adding rare gas atom attachment and dissociation rates
    attachmentRate0 = f" - k31_0*{nHe}**2*{N[0]} + kCID1*{nHe}*{N_He[0]}*p"
    attachmentRate1 = f" - k31_1*{nHe}**2*{N[1]} + kCID1*{nHe}*{N_He[0]}*(1-p)"
    attachmentRate2 = f" - k32*{nHe}**2*{N_He[0]} + kCID2*{nHe}*{N_He[1]}"
    
    dR_dt[0] += attachmentRate0
    dR_dt[1] += attachmentRate1
    
    # CDHe:
    dCDHe_dt = attachmentRate0 + attachmentRate1 + attachmentRate2
    dR_dt.append(dCDHe_dt)
    
    # CDHe2
    dCDHe2_dt = f" - ({attachmentRate2})"
    dR_dt.append(dCDHe2_dt)
    
    for i, eq in enumerate(dR_dt):
        if i>=totallevel:
            fullRateEquation.append(f"NHe{i} = {eq}")
        else:
            fullRateEquation.append(f"N{i} = {eq}")
            
    return dR_dt, fullRateEquation


conditions = {
    "totallevel" : totallevel,
    "includeSpontaneousEmissionForAllLevels" : True,
    "excitedLevel" : 1,
    "lightON" : False,
    "nHe" : "nHe"
}

N, fullRateEquation = showRateDistributionEquations(**conditions)
fullRateEquation

In [ ]:
totalLevel = 5
q_excitation = []
def get_level_qRate(totallevel=6, samelevelExistInFile=True):
    q_rate_deexcitation = []
    counter = 0
    for n in range(1, totallevel):
        for _temp in range(n+1):
            if _temp != n: 
                q_rate_deexcitation.append(f"{n}{_temp}")
                if not samelevelExistInFile: counter += 1
            if samelevelExistInFile: counter += 1
    
    return q_rate_deexcitation